In [1]:
from utils import UNet, SegmentationDataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

dataset = SegmentationDataset("SA1B_Meta_AI_Segmentation_Dataset/")
dataloader = DataLoader(dataset, batch_size=50, shuffle=True)
model = UNet(in_channels=3, out_channels=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
def train_unet(model, dataset, epochs=10, lr=1e-4):
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()  # For binary segmentation

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0.0
        for imgs, masks in dataloader:
            imgs, masks = imgs.cuda(), masks.cuda()  # Move to GPU if available

            # Forward pass
            preds = model(imgs)
            loss = criterion(preds, masks.unsqueeze(1))  # Add channel dim to masks
            epoch_loss += loss.item()

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(dataloader)}")

    return model

In [3]:
model.train()
for epoch in range(2):
	epoch_loss = 0
	for idx, (image, mask) in enumerate(dataloader):
		outputs = model(image)
		loss = criterion(outputs, mask)
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
		epoch_loss += loss.item()
		# outputs.weights
		print(f'epoch: {epoch+1} | loss: {epoch_loss}')


Bounding Box: [1136.0, 960.0, 31.0, 163.0]
Segmentation RLE: VQQ^28QV25K6J7flMi0Uo1[3J2O1N100000O01000001O00000...
Predicted IoU: 0.9608914256095886
Bounding Box: [226.0, 2155.0, 60.0, 52.0]
Segmentation RLE: Ufb?6RV26K8H8H8I2N2M2O100O1O001O1O001O1O00001O0000...
Predicted IoU: 0.9560363292694092
Bounding Box: [71.0, 2173.0, 53.0, 44.0]
Segmentation RLE: hTn44mU2<J4M4M2M4N3L2N3N001N101O00001O001O001N1000...
Predicted IoU: 0.9678314328193665
Bounding Box: [140.0, 2186.0, 71.0, 62.0]
Segmentation RLE: ehe9:jU2:J4L4M4L3N1N2N1O2N2O1N101O0O2O001O1O001O00...
Predicted IoU: 0.9682207107543945
Bounding Box: [201.0, 2059.0, 55.0, 59.0]
Segmentation RLE: lek=;kU27G7N3M3N2M3N2M2N2N2O1N2N101O0O2O0O2O001O0O...
Predicted IoU: 0.977738618850708
Bounding Box: [0.0, 2157.0, 61.0, 45.0]
Segmentation RLE: ^S2o0ZU22O2N1O1O000001O0001O00010O1O0010O01O1O01O0...
Predicted IoU: 0.9079374074935913
Bounding Box: [0.0, 1797.0, 62.0, 41.0]
Segmentation RLE: ch1g0cU20O10000O10000O100O010O01N1O2N2N2L5JZW2IohM...
Pre

TypeError: object of type 'builtin_function_or_method' has no len()

In [ ]:
print(dataset)